In [79]:
import pandas as pd
import numpy as np
import nltk

In [80]:
testset=pd.read_csv('edited_city_list.csv')  

In [81]:
dataset=pd.read_csv('Town and Counties by Postcode - Sheet1.csv')

In [84]:
testcode=testset['postcode'].fillna('').apply(lambda s: s.split()[0] if len(s)>0 else '')

In [85]:
testcity=testset['city']
testcity=testcity.fillna('')

In [86]:
dcode=dataset['Postcode districts']
dtown=dataset['Post Town']

In [87]:
testset['predicted']='null'
testset['successful']='0'
testset

,city,postcode,Actual,predicted,successful
0,Chennai,600096,Wrong Input,null,0
1,Aberdeen City,AB10 6QD,Aberdeen,null,0
2,-,AB11 9LX,Aberdeen,null,0
3,Petercult,AB165QN,Peterculter,null,0
4,Abedeen,AB24 2RN,Aberdeen,null,0
5,NaN,AB24 5AE,Aberdeen,null,0
6,Aberdeenshire,AB24 5PS,Aberdeen,null,0
7,NaN,AB33 8EW,Alford,null,0
8,Aberdeenshire,AB42 1TN,Peterhead,null,0
9,Aberdeen,AB43 9AS,Fraserburgh,null,0


In [88]:

for j in range(len(testcity)):  # Loop for the test set
    flagp=0       # right postcode match hasn't been found
    flagr=0       # right city match hasn't been found
    small=100  
    for i in range(len(dataset)):    # Loop for the data set
        if (len(testcode[j])!=0):    # to make sure the postcode entry isn't empty
            code=(dcode[i].replace('\n','')).split(', ')   # Incase of a cell with many postcode separate them and check
            for l in range(len(code)):                       
                p=nltk.edit_distance(code[l], testcode[j]) 
                if(p==0):        
                    flagp=1     #exact postcode match has been found
                    testset['predicted'].iat[j]=dtown[i]   
                    break
    
    if(flagp==0):      #If the given postcode is wrong or entry is empty
        for i in range(len(dataset)):    #Loop for the data set
            if (len(testcity[j])!=0):    #make sure city entry isn't empty
                r=nltk.edit_distance((testcity[j].lower()),(dtown[i].lower()))  #check for the distance
                if(r<small and (r/(len(testcity[j])))<0.4):  #to find the smallest distance and make sure it's below the threshold value  
                    flagr=1      #right city match has been found
                    small=r
                    pos=i
                    
    if(flagr==1 and flagp==0):     #postcode is wrong and city given is right
        testset['predicted'].iat[j]=dtown[pos]
            
    if (flagp==0 and flagr==0):    #both the given postcode and city name are wrong
        testset['predicted'].iat[j]=("Wrong Input")
    


In [89]:
for k in range(len(testcity)):    #to check if the predicted output and the actual output are same
    e=nltk.edit_distance(testset['Actual'].iat[k].lower(),testset['predicted'].iat[k].lower())
    if(e==0):
        testset['successful'].iat[k]='1'

In [90]:
testset

,city,postcode,Actual,predicted,successful
0,Chennai,600096,Wrong Input,Wrong Input,1
1,Aberdeen City,AB10 6QD,Aberdeen,Aberdeen,1
2,-,AB11 9LX,Aberdeen,Aberdeen,1
3,Petercult,AB165QN,Peterculter,Peterculter,1
4,Abedeen,AB24 2RN,Aberdeen,Aberdeen,1
5,NaN,AB24 5AE,Aberdeen,Aberdeen,1
6,Aberdeenshire,AB24 5PS,Aberdeen,Aberdeen,1
7,NaN,AB33 8EW,Alford,Alford,1
8,Aberdeenshire,AB42 1TN,Peterhead,Peterhead,1
9,Aberdeen,AB43 9AS,Fraserburgh,Fraserburgh,1


In [91]:
testset.to_csv('prediction.csv')

In [93]:
correct=0
for i in range(len(testcity)):
    if(testset['successful'].iat[i]=='1'):
        correct=correct+1
accuracy = (correct/len(testcity))*100
print("No. of correct predictions =",correct)
print("Accuracy of prediction =",accuracy,"%")  

No. of correct predictions = 1914
Accuracy of prediction = 97.55351681957187 %
